In [ ]:
import glob, os
import ast
import collections
import json
from pymongo import MongoClient
from bson.objectid import ObjectId

In [ ]:
client = MongoClient('mongodb://######################/')
db = client['SONAR_CHALLENGE']
collection = db["python_repositories_metadata"]

In [ ]:
def get_source(python_file):
    
    source_code = None
    
    with open(os.path.join(python_file), "rb") as f:
        source_code = f.read()
        f.close()
        
    return source_code

In [ ]:
def get_frequency(syntax_elements):

    return dict(collections.Counter(syntax_elements))

In [ ]:
def get_syntax_elements(python_file):
    
    source_code = get_source(python_file)

    if source_code != None:
        tree = ast.parse(source_code)
        elements = [child.__class__.__name__ for node in ast.walk(tree) for child in ast.iter_child_nodes(node) if isinstance(child, ast.AST)]
        return elements

In [ ]:
def processing_files():
           
    try:
        
        cursors = collection.find({'clone':True,'syntax':False})
        
        for repo in cursors:

            projects_folder_path = f'./repositories/{repo["full_name"].replace("/", "__")}'

            files = []

            for file_name in glob.iglob(f'{projects_folder_path}/**', recursive=True):
                if os.path.isfile(file_name):
                    if file_name.endswith('.py') and 'test' not in file_name:

                        data_file = {}  
                        data_file['file_name'] = file_name
                        
                        try:
                            data_file['syntax_elements_in_file'] = get_frequency(get_syntax_elements(file_name))
                            files.append(data_file)
                        except:
                            pass
                        
            collection.update_one({'_id': ObjectId(repo['_id'])},
                                       {"$set": 
                                        {
                                            'syntax_for_each_file' : files,
                                            'syntax': True
                                        }
                                       })
                        
    except Exception as e:
        
        print(e)
        collection.update_one({'_id': ObjectId(repo['_id'])},
                       {"$set": 
                        {
                            'syntax': False
                        }
                       })




In [ ]:
def main():
    processing_files()

if __name__ == "__main__":
    main()